# Notebook pro trénink s destilací nad datasetem CIFAR10
V tomto notebooku je trénován MobileNetV2 nad datasetem CIFAR10, jako učitelsý model je využíván finetunued ViT nad stejným datasetem. 

MobileNetV2 je používán s náhodnou inicializací, tréninkem pouze klasifikační hlavy inicializovaného (předtrénovaného nad ImageNetem) MobileNetuV2 a trénink celého modelu, taktéž inicializovaného. Tyto tři úlohy jsou trénovány bězným způsobem a také s pomocí destilace výše zmíněného modelu.  

Při destilaci je využíváno předpočítaných logitů ze sešitu precompute_logits.

## Import knihoven a definice metod

In [1]:
from transformers import Trainer, EarlyStoppingCallback
from torch.utils.data import DataLoader
import pandas as pd
import torch
import base
import os

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [2]:
dataset_part = base.get_dataset_part()

Resetování náhodného seedu pro replikovatelnost výsledků.

In [3]:
base.reset_seed()

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


Provedení transformací nad datasetem.

In [5]:
DATASET = "cifar10"

In [6]:
transform = base.base_transforms()

#Poslední train batch použijeme jako eval část...
test = base.CustomCIFAR10L(root=f"{os.path.expanduser('~')}/data/10-logits", dataset_part=dataset_part.TEST, transform=transform, device="cpu")
train = base.CustomCIFAR10L(root=f"{os.path.expanduser('~')}/data/10-logits", dataset_part=dataset_part.TRAIN, transform=transform, device="cpu")
eval = base.CustomCIFAR10L(root=f"{os.path.expanduser('~')}/data/10-logits", dataset_part=dataset_part.EVAL, transform=transform, device="cpu")

In [7]:
train[0]["labels"]

tensor(6)

In [8]:
# Test rozložení --> Good Enough
df = pd.DataFrame(eval.labels)
print(df.value_counts())

0
5    1025
9    1022
3    1016
0    1014
1    1014
8    1003
4     997
6     980
7     977
2     952
Name: count, dtype: int64


In [9]:
train_part_cpu = base.CustomCIFAR10(root=f"{os.path.expanduser('~')}/data/10", train=True, batch=1, transform=transform, device="cpu")
cpu_data_loader = DataLoader(train_part_cpu, batch_size=1, shuffle=False)
train_part_gpu = base.CustomCIFAR10(root=f"{os.path.expanduser('~')}/data/10", train=True, batch=1, transform=transform, device="cuda")
gpu_data_loader = DataLoader(train_part_gpu, batch_size=1, shuffle=False)

### Standardní trénink náhodně inicializovaného modelu. 

In [10]:
base.reset_seed()

In [11]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/cifar10-random", logging_dir=f"~/logs/{DATASET}/cifar10-random", lr=0.0006, weight_decay=0.005, warmup_steps=25, epochs=20)
model = base.get_random_init_mobilenet(10)

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [14]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.595100,1.237562,0.559400,0.574408,0.559382,0.547235
2,1.024000,0.871471,0.690300,0.696350,0.690017,0.689767
3,0.802900,0.788958,0.735000,0.736337,0.734403,0.730367
4,0.668500,0.600232,0.790900,0.793263,0.790918,0.789307
5,0.551100,0.581511,0.803500,0.810439,0.803278,0.804223
6,0.469800,0.589092,0.794400,0.808125,0.794034,0.794595
7,0.386700,0.567914,0.818100,0.822736,0.818610,0.815618
8,0.321600,0.507972,0.831500,0.835464,0.831681,0.831222
9,0.260300,0.663696,0.808200,0.830179,0.807736,0.807199
10,0.196400,0.535166,0.840400,0.842509,0.840813,0.838381


TrainOutput(global_step=6260, training_loss=0.3380684549149614, metrics={'train_runtime': 1569.3447, 'train_samples_per_second': 509.767, 'train_steps_per_second': 3.989, 'total_flos': 1.6160796868608e+18, 'train_loss': 0.3380684549149614, 'epoch': 20.0})

In [15]:
model.eval()

MobileNetV2ForImageClassification(
  (mobilenet_v2): MobileNetV2Model(
    (conv_stem): MobileNetV2Stem(
      (first_conv): MobileNetV2ConvLayer(
        (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (conv_3x3): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (reduce_1x1): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (normalization): BatchNorm2d(16, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
      )
    )
    (layer): ModuleList(
      (0): MobileNetV2InvertedResidual(
        (expand_1x1): MobileNe

In [16]:
trainer.evaluate(test)

{'eval_loss': 0.7480902075767517,
 'eval_accuracy': 0.8519,
 'eval_precision': 0.8549866485308314,
 'eval_recall': 0.8519,
 'eval_f1': 0.8519241289796995,
 'eval_runtime': 12.6152,
 'eval_samples_per_second': 792.692,
 'eval_steps_per_second': 6.262,
 'epoch': 20.0}

In [17]:
torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/random-base.pth")

## Definice destilačního tréninku

Třída, která upravuje hugging face trenéra pro destilaci znalostí. Nově pracuje s logity uloženými v datasetu.

### Trénink náhodně inicializovaného modelu s pomocí destilace znalostí

In [21]:
base.reset_seed()

In [22]:
student_model = base.get_random_init_mobilenet(10)

In [23]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/cifar10-random-KD", logging_dir=f"~/logs/{DATASET}/cifar10-random-KD", remove_unused_columns=False, epochs=20, lr=0.00065, weight_decay=0.008, warmup_steps=5, lambda_param=.4, temp=6.5)

In [24]:
trainer = base.DistilTrainer(
    student_model=student_model,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [25]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.211600,0.923582,0.581900,0.604312,0.582070,0.574919
2,0.817700,0.690882,0.705600,0.708314,0.705495,0.700665
3,0.658700,0.639006,0.740700,0.748375,0.739517,0.737840
4,0.565500,0.529884,0.788100,0.794764,0.787768,0.788780
5,0.497800,0.506643,0.800300,0.806032,0.800161,0.800127
6,0.435300,0.506764,0.800400,0.813745,0.800562,0.800548
7,0.386300,0.484542,0.816500,0.824948,0.817161,0.813032
8,0.342400,0.437639,0.841000,0.844455,0.841260,0.840168
9,0.307500,0.485153,0.818000,0.835350,0.817510,0.817941
10,0.268400,0.424161,0.846400,0.851865,0.846942,0.844988


TrainOutput(global_step=6260, training_loss=0.3607652871372601, metrics={'train_runtime': 1517.4651, 'train_samples_per_second': 527.195, 'train_steps_per_second': 4.125, 'total_flos': 1.6160796868608e+18, 'train_loss': 0.3607652871372601, 'epoch': 20.0})

In [26]:
student_model.eval()

MobileNetV2ForImageClassification(
  (mobilenet_v2): MobileNetV2Model(
    (conv_stem): MobileNetV2Stem(
      (first_conv): MobileNetV2ConvLayer(
        (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (conv_3x3): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (reduce_1x1): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (normalization): BatchNorm2d(16, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
      )
    )
    (layer): ModuleList(
      (0): MobileNetV2InvertedResidual(
        (expand_1x1): MobileNe

In [27]:
trainer.evaluate(test)

{'eval_loss': 0.38627079129219055,
 'eval_accuracy': 0.8581,
 'eval_precision': 0.8618272904669393,
 'eval_recall': 0.8581000000000001,
 'eval_f1': 0.8588283097517347,
 'eval_runtime': 13.2198,
 'eval_samples_per_second': 756.439,
 'eval_steps_per_second': 5.976,
 'epoch': 20.0}

In [ ]:
torch.save(student_model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/random-distill.pth")

## Získání inicializovaného MobileNetV2 modelu

In [11]:
base.reset_seed()

In [12]:
model_pretrained = base.get_mobilenet(10)

In [13]:
print(model_pretrained)

MobileNetV2ForImageClassification(
  (mobilenet_v2): MobileNetV2Model(
    (conv_stem): MobileNetV2Stem(
      (first_conv): MobileNetV2ConvLayer(
        (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (conv_3x3): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (reduce_1x1): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (normalization): BatchNorm2d(16, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
      )
    )
    (layer): ModuleList(
      (0): MobileNetV2InvertedResidual(
        (expand_1x1): MobileNe

In [14]:
model_pretrained = base.freeze_model(model_pretrained)

In [16]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/cifar10-pretrained-head", logging_dir=f"~/logs/{DATASET}/cifar10-pretrained-head", epochs=20, lr=0.0025, weight_decay=0.005, warmup_steps=10)

In [17]:
trainer = Trainer(
    model=model_pretrained,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [18]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.874200,0.760877,0.736300,0.759207,0.736365,0.737450
2,0.723500,0.722264,0.749400,0.758672,0.748687,0.748584
3,0.699600,0.739107,0.747200,0.761697,0.746236,0.745191
4,0.695500,0.679236,0.766600,0.777536,0.766793,0.767195
5,0.686300,0.682635,0.759800,0.763131,0.759497,0.760271
6,0.681900,0.676907,0.763900,0.770916,0.763320,0.764351
7,0.672000,0.699199,0.763400,0.768724,0.763841,0.759322


TrainOutput(global_step=2191, training_loss=0.7190116976342317, metrics={'train_runtime': 365.6388, 'train_samples_per_second': 2187.952, 'train_steps_per_second': 17.121, 'total_flos': 5.6562789040128e+17, 'train_loss': 0.7190116976342317, 'epoch': 7.0})

In [19]:
model_pretrained.eval()

MobileNetV2ForImageClassification(
  (mobilenet_v2): MobileNetV2Model(
    (conv_stem): MobileNetV2Stem(
      (first_conv): MobileNetV2ConvLayer(
        (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (conv_3x3): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (reduce_1x1): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (normalization): BatchNorm2d(16, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
      )
    )
    (layer): ModuleList(
      (0): MobileNetV2InvertedResidual(
        (expand_1x1): MobileNe

In [20]:
trainer.evaluate(test)

{'eval_loss': 0.6907253265380859,
 'eval_accuracy': 0.7613,
 'eval_precision': 0.772018627546524,
 'eval_recall': 0.7613,
 'eval_f1': 0.7617999755057994,
 'eval_runtime': 13.1351,
 'eval_samples_per_second': 761.32,
 'eval_steps_per_second': 6.014,
 'epoch': 7.0}

In [21]:
torch.save(model_pretrained.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/head-base.pth")

### Trénink inicializovaného MobileNetV2

In [22]:
base.reset_seed()

In [23]:
model_pretrained_whole = base.get_mobilenet(10)

In [24]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/cifar10-pretrained", logging_dir=f"~/logs/{DATASET}/cifar10-pretrained", epochs=20, lr=0.00045, weight_decay=0.008, warmup_steps=3)

In [25]:
trainer = Trainer(
    model=model_pretrained_whole,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [26]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.402600,0.316462,0.896600,0.906195,0.896590,0.897089
2,0.169600,0.262278,0.918200,0.923636,0.918248,0.919566
3,0.108700,0.267307,0.918000,0.919754,0.918078,0.917876
4,0.078400,0.293605,0.918200,0.921895,0.918528,0.918707
5,0.055600,0.270866,0.925200,0.926872,0.925615,0.925130
6,0.043400,0.272729,0.931200,0.932300,0.931321,0.931164
7,0.031900,0.325340,0.926100,0.927471,0.926519,0.925840
8,0.027400,0.294779,0.934800,0.936635,0.935011,0.934850
9,0.019600,0.384792,0.922900,0.928752,0.922822,0.923312
10,0.015700,0.284001,0.937300,0.938128,0.937573,0.937226


TrainOutput(global_step=6260, training_loss=0.0494139636119905, metrics={'train_runtime': 1500.3325, 'train_samples_per_second': 533.215, 'train_steps_per_second': 4.172, 'total_flos': 1.6160796868608e+18, 'train_loss': 0.0494139636119905, 'epoch': 20.0})

In [27]:
model_pretrained_whole.eval()

MobileNetV2ForImageClassification(
  (mobilenet_v2): MobileNetV2Model(
    (conv_stem): MobileNetV2Stem(
      (first_conv): MobileNetV2ConvLayer(
        (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (conv_3x3): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (reduce_1x1): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (normalization): BatchNorm2d(16, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
      )
    )
    (layer): ModuleList(
      (0): MobileNetV2InvertedResidual(
        (expand_1x1): MobileNe

In [28]:
trainer.evaluate(test)

{'eval_loss': 0.2871420085430145,
 'eval_accuracy': 0.9483,
 'eval_precision': 0.949222638232329,
 'eval_recall': 0.9483,
 'eval_f1': 0.9482737554304345,
 'eval_runtime': 12.3137,
 'eval_samples_per_second': 812.103,
 'eval_steps_per_second': 6.416,
 'epoch': 20.0}

In [29]:
torch.save(model_pretrained_whole.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/pretrained-base.pth")

## Trénink s pomocí destilace znalostí inicializovaného MobileNetV2

### Trénink inicializovaného modelu - pouze klasifikační hlavy s pomocí destilace

In [30]:
base.reset_seed()

In [31]:
student_model_pretrained = base.get_mobilenet(10)

In [32]:
student_model_pretrained = base.freeze_model(student_model_pretrained)

In [33]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/cifar10-pretrained-head-KD", logging_dir=f"~/logs/{DATASET}/cifar10-pretrained-head-KD", remove_unused_columns=False, epochs=20, lr=0.00065, weight_decay=.008, lambda_param=.6, temp=3.5)

In [34]:
trainer = base.DistilTrainer(
    student_model=student_model_pretrained,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [35]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.738900,0.641624,0.729000,0.751844,0.728723,0.729765
2,0.629700,0.617656,0.744700,0.750206,0.744170,0.743413
3,0.616300,0.617649,0.751300,0.757247,0.750454,0.750100
4,0.611900,0.594144,0.759800,0.767750,0.759944,0.759688
5,0.608300,0.606135,0.747100,0.756662,0.746875,0.749526
6,0.606700,0.592881,0.761000,0.764229,0.760641,0.760734
7,0.603900,0.603333,0.760100,0.761047,0.760031,0.755976
8,0.600800,0.606672,0.756600,0.766292,0.756417,0.756099
9,0.602300,0.600940,0.758500,0.763569,0.758112,0.758541


TrainOutput(global_step=2817, training_loss=0.6243223678485951, metrics={'train_runtime': 470.0682, 'train_samples_per_second': 1701.881, 'train_steps_per_second': 13.317, 'total_flos': 7.2723585908736e+17, 'train_loss': 0.6243223678485951, 'epoch': 9.0})

In [36]:
student_model_pretrained.eval()

MobileNetV2ForImageClassification(
  (mobilenet_v2): MobileNetV2Model(
    (conv_stem): MobileNetV2Stem(
      (first_conv): MobileNetV2ConvLayer(
        (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (conv_3x3): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (reduce_1x1): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (normalization): BatchNorm2d(16, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
      )
    )
    (layer): ModuleList(
      (0): MobileNetV2InvertedResidual(
        (expand_1x1): MobileNe

In [37]:
trainer.evaluate(test)

{'eval_loss': 0.5939173102378845,
 'eval_accuracy': 0.762,
 'eval_precision': 0.7652446763292213,
 'eval_recall': 0.7619999999999999,
 'eval_f1': 0.7621432153880755,
 'eval_runtime': 13.2693,
 'eval_samples_per_second': 753.619,
 'eval_steps_per_second': 5.954,
 'epoch': 9.0}

In [38]:
torch.save(student_model_pretrained.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/head-distill.pth")

### Trénink inicializovaného modelu s pomocí destilace

In [39]:
base.reset_seed()

In [40]:
student_model_pretrained_whole = base.get_mobilenet(10)

In [41]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/cifar10-pretrained-KD", logging_dir=f"~/logs/{DATASET}/cifar10-pretrained-KD", remove_unused_columns=False, epochs=20, lr=0.00055, weight_decay=.005, lambda_param=.3, temp=2.5)

In [ ]:
trainer = base.DistilTrainer(
    student_model=student_model_pretrained_whole,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [43]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.423800,0.395326,0.885700,0.899943,0.885494,0.886858
2,0.260300,0.311617,0.916900,0.921539,0.917047,0.917589
3,0.213700,0.282169,0.929000,0.929931,0.929109,0.928981
4,0.188300,0.270789,0.931200,0.934023,0.931358,0.931556
5,0.168900,0.256467,0.936200,0.937201,0.936470,0.936110
6,0.159700,0.267777,0.930900,0.933351,0.930871,0.931282
7,0.148800,0.272582,0.932300,0.934309,0.932945,0.931894
8,0.144700,0.247279,0.942200,0.944623,0.942323,0.942457
9,0.142100,0.265452,0.936600,0.939780,0.936683,0.936808
10,0.137700,0.236977,0.946900,0.948278,0.947181,0.946943


TrainOutput(global_step=4382, training_loss=0.17989203114859975, metrics={'train_runtime': 1050.1591, 'train_samples_per_second': 761.789, 'train_steps_per_second': 5.961, 'total_flos': 1.13125578080256e+18, 'train_loss': 0.17989203114859975, 'epoch': 14.0})

In [44]:
student_model_pretrained_whole.eval()

MobileNetV2ForImageClassification(
  (mobilenet_v2): MobileNetV2Model(
    (conv_stem): MobileNetV2Stem(
      (first_conv): MobileNetV2ConvLayer(
        (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (conv_3x3): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
        (normalization): BatchNorm2d(32, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
        (activation): ReLU6()
      )
      (reduce_1x1): MobileNetV2ConvLayer(
        (convolution): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (normalization): BatchNorm2d(16, eps=0.001, momentum=0.997, affine=True, track_running_stats=True)
      )
    )
    (layer): ModuleList(
      (0): MobileNetV2InvertedResidual(
        (expand_1x1): MobileNe

In [45]:
trainer.evaluate(test)

{'eval_loss': 0.2267916053533554,
 'eval_accuracy': 0.9489,
 'eval_precision': 0.9489642665551571,
 'eval_recall': 0.9489000000000001,
 'eval_f1': 0.9487862446825236,
 'eval_runtime': 12.5707,
 'eval_samples_per_second': 795.499,
 'eval_steps_per_second': 6.284,
 'epoch': 14.0}

In [46]:
torch.save(student_model_pretrained_whole.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/pretrained-distill.pth")

In [47]:
base.count_parameters(student_model_pretrained_whole)

model size: 8.663MB.
Total Trainable Params: 2236682.


,Modules,Parameters
0,mobilenet_v2.conv_stem.first_conv.convolution....,864
1,mobilenet_v2.conv_stem.first_conv.normalizatio...,32
2,mobilenet_v2.conv_stem.first_conv.normalizatio...,32
3,mobilenet_v2.conv_stem.conv_3x3.convolution.we...,288
4,mobilenet_v2.conv_stem.conv_3x3.normalization....,32
...,...,...
153,mobilenet_v2.conv_1x1.convolution.weight,409600
154,mobilenet_v2.conv_1x1.normalization.weight,1280
155,mobilenet_v2.conv_1x1.normalization.bias,1280
156,classifier.weight,12800


In [48]:
base.count_parameters(student_model_pretrained)

model size: 8.663MB.
Total Trainable Params: 12810.


,Modules,Parameters
0,classifier.weight,12800
1,classifier.bias,10


In [49]:
cpu_benchmark = base.BenchMarkRunner(student_model_pretrained_whole, cpu_data_loader, "cpu", 1000)
print(cpu_benchmark.run_benchmark())

self.infer_speed_comp()
  22.01 ms
  1 measurement, 1000 runs , 4 threads


In [50]:
gpu_benchmark = base.BenchMarkRunner(student_model_pretrained_whole, gpu_data_loader, "cuda", 1000)
print(gpu_benchmark.run_benchmark())

self.infer_speed_comp()
  8.72 ms
  1 measurement, 1000 runs , 4 threads
